# Matrix testing

In [4]:
from scipy.sparse import load_npz

similarity_ascents = load_npz("similarity_ascent.npz")
similarity_style = load_npz("similarity_style.npz")
similarity_grade = load_npz("similarity_grade.npz")

In [6]:
import numpy as np

def recommend_boulders(
    input_boulders, top_n=5, alpha=0.5, beta=0.25, gamma=0.25
):

    ascents = similarity_ascents[:, input_boulders].sum(axis=1).A1
    style = similarity_style[:, input_boulders].sum(axis=1).A1
    grade = similarity_grade[:, input_boulders].sum(axis=1).A1

    ascents[input_boulders] = 0
    style[input_boulders] = 0
    grade[input_boulders] = 0

    sim_scores = alpha * ascents + beta * style + gamma * grade

    best_boulders = np.argsort(-sim_scores)[:top_n]

    return best_boulders.tolist()


recommend_boulders([60], top_n=10)

[1837, 4641, 28325, 8828, 25143, 11405, 3882, 1835, 30911, 28318]